# Smart Parser

Ziel ist ein Parser, der einfache Sätze besser versteht. Eingaben sind gewöhnliche Text-Adventure-Sätze, Ausgaben sind die Commands.

In [ ]:
import spacy 

model_lg = spacy.load("de_core_news_lg")
model_trf = spacy.load("de_dep_news_trf")

doc_lg = model_lg("Nimm den goldenen Schlüssel und öffne die verzauberte Truhe")
# doc_trf = model_trf("Nimm den goldenen Schlüssel und öffne die verzauberte Truhe")

In [3]:

spacy.displacy.render(doc_lg, style='dep', jupyter=True)
# spacy.displacy.render(doc_trf, style='dep', jupyter=True)

In [ ]:
satz = None

for token in satz:
    print(f"{token.text:15} | POS: {token.pos_:6} | DEP: {token.dep_:10} | LEMMA: {token.lemma_:15} | HEAD: {token.head.text}")

Nimm            | POS: VERB   | DEP: ROOT       | LEMMA: Nimm            | HEAD: Nimm
den             | POS: DET    | DEP: nk         | LEMMA: der             | HEAD: Schlüssel
goldenen        | POS: ADJ    | DEP: nk         | LEMMA: golden          | HEAD: Schlüssel
Schlüssel       | POS: NOUN   | DEP: oa         | LEMMA: Schlüssel       | HEAD: Nimm
und             | POS: CCONJ  | DEP: cd         | LEMMA: und             | HEAD: Nimm
öffne           | POS: VERB   | DEP: cj         | LEMMA: öffnen          | HEAD: und
die             | POS: DET    | DEP: nk         | LEMMA: der             | HEAD: Truhe
verzauberte     | POS: ADJ    | DEP: nk         | LEMMA: verzaubert      | HEAD: Truhe
Truhe           | POS: NOUN   | DEP: oa         | LEMMA: Truhe           | HEAD: öffne


In [5]:
basic = [
    "Nimm den Schlüssel",
    "Geh zur Taverne",
    "Zeige das Inventar",
    "Öffne die Tür",
    "Lege das Schwert ab"
]

trennbar = [
    "Nimm den Schlüssel auf",
    "Hebe das Gold auf",
    "Wirf den Apfel weg",
    "Steck das Messer ein",
    "Pack die Fackel ein"
]

komplex = [
    "Nimm den goldenen Schlüssel",
    "Hole das rostige Schwert",
    "Geh in die dunkle Taverne",
    "Nimm den kleinen roten Apfel auf",
    "Lege das schwere eiserne Schwert ab"
]

praepositionen = [
    "Hole den Schlüssel aus der Truhe",
    "Gehe in die Taverne",
    "Lege das Schwert auf den Tisch",
    "Nimm das Buch vom Regal",
    "Stecke den Schlüssel in die Tasche"
]

synonyme = [
    "Hol den Schlüssel",           # hol = nimm
    "Pack den Apfel",              # pack = nimm
    "Greif nach dem Schwert",      # greif = nimm
    "Besuche die Taverne",         # besuche = visit
    "Lauf zur Taverne",            # lauf = visit
    "Bewege dich zum Marktplatz"   # bewege = visit
]

schwierig = [
    "Schnapp dir den Schlüssel",
    "Mach dass du zur Taverne kommst",
    "Lass das Schwert fallen",
    "Schau dir das Inventar an",
    "Was habe ich alles dabei?",
    "Wo bin ich gerade?",
    "Nimm alles mit was hier liegt"
]

edge_cases = [
    "Nimm Schlüssel",              # Ohne Artikel
    "den Schlüssel nehmen",        # Infinitiv statt Imperativ
    "Ich möchte den Schlüssel nehmen",  # Modalverb
    "Kannst du mir das Inventar zeigen?",  # Frage
    "Schlüssel",                   # Nur ein Wort
    ""                             # Leerer String
]

all_tests = {
    'basic': basic,
    'trennbar': trennbar,
    'komplex': komplex,
    'praepositionen': praepositionen,
    'synonyme': synonyme,
    'schwierig': schwierig,
    'edge_cases': edge_cases
}

In [9]:
for category, sentences in all_tests.items():
    print(f"\n{'='*60}")
    print(f"KATEGORIE: {category}")
    print('='*60)

    for sentence in sentences:
        doc = model_lg(sentence)
        print(f"\n'{sentence}':")
        for token in doc:
            print(f"{token.text:15} | POS: {token.pos_:6} | DEP: {token.dep_:10} | LEMMA: {token.lemma_:15} | HEAD: {token.head.text}")


KATEGORIE: basic

'Nimm den Schlüssel':
Nimm            | POS: VERB   | DEP: ROOT       | LEMMA: Nimm            | HEAD: Nimm
den             | POS: DET    | DEP: nk         | LEMMA: der             | HEAD: Schlüssel
Schlüssel       | POS: NOUN   | DEP: oa         | LEMMA: Schlüssel       | HEAD: Nimm

'Geh zur Taverne':
Geh             | POS: ADV    | DEP: ROOT       | LEMMA: Geh             | HEAD: Geh
zur             | POS: ADP    | DEP: mnr        | LEMMA: zu              | HEAD: Geh
Taverne         | POS: NOUN   | DEP: nk         | LEMMA: Taverne         | HEAD: zur

'Zeige das Inventar':
Zeige           | POS: VERB   | DEP: ROOT       | LEMMA: zeige           | HEAD: Zeige
das             | POS: DET    | DEP: nk         | LEMMA: der             | HEAD: Inventar
Inventar        | POS: NOUN   | DEP: pd         | LEMMA: Inventar        | HEAD: Zeige

'Öffne die Tür':
Öffne           | POS: VERB   | DEP: ROOT       | LEMMA: Öffne           | HEAD: Öffne
die             | POS: DET   

In [15]:
for category, sentences in all_tests.items():
    print(f"\n{'='*60}")
    print(f"KATEGORIE: {category}")
    print('='*60)

    for sentence in sentences:
        lg_doc = model_lg(sentence)
        lg_root_verb = None
        lg_objects = []
        for token in lg_doc:
            # Hauptverb finden
            if token.dep_ == "ROOT":
                lg_root_verb = token
            # Objekte finden
            if token.dep_ in ['obj', 'dobj', 'oa', 'pobj']:
                lg_objects.append(token)
        
        trf_doc = model_trf(sentence)
        trf_root_verb = None
        trf_objects = []
        for token in trf_doc:
            # Hauptverb finden
            if token.dep_ == "ROOT":
                trf_root_verb = token
            # Objekte finden
            if token.dep_ in ['obj', 'dobj', 'oa', 'pobj']:
                trf_objects.append(token)
        


        print(f"{sentence}")
        print(f" Verb: {lg_root_verb} | Objekte: {lg_objects}")
        print(f" Verb: {trf_root_verb} | Objekte: {trf_objects}")




KATEGORIE: basic
Nimm den Schlüssel
 Verb: Nimm | Objekte: [Schlüssel]
 Verb: Nimm | Objekte: [Schlüssel]
Geh zur Taverne
 Verb: Geh | Objekte: []
 Verb: Geh | Objekte: []
Zeige das Inventar
 Verb: Zeige | Objekte: []
 Verb: Zeige | Objekte: [Inventar]
Öffne die Tür
 Verb: Öffne | Objekte: [Tür]
 Verb: Öffne | Objekte: [Tür]
Lege das Schwert ab
 Verb: Lege | Objekte: [Schwert]
 Verb: Lege | Objekte: [Schwert]

KATEGORIE: trennbar
Nimm den Schlüssel auf
 Verb: Nimm | Objekte: [Schlüssel]
 Verb: Nimm | Objekte: [Schlüssel]
Hebe das Gold auf
 Verb: Gold | Objekte: []
 Verb: Hebe | Objekte: [Gold]
Wirf den Apfel weg
 Verb: Wirf | Objekte: [Apfel]
 Verb: Wirf | Objekte: [Apfel]
Steck das Messer ein
 Verb: Steck | Objekte: []
 Verb: Steck | Objekte: [Messer]
Pack die Fackel ein
 Verb: ein | Objekte: []
 Verb: Pack | Objekte: [Fackel]

KATEGORIE: komplex
Nimm den goldenen Schlüssel
 Verb: Nimm | Objekte: [Schlüssel]
 Verb: Nimm | Objekte: [Schlüssel]
Hole das rostige Schwert
 Verb: Hole | Ob